In [1]:
from einops import rearrange

import autograd.numpy as np
from autograd import jacobian
from autograd import elementwise_grad
from autograd import grad

import logging
from copy import deepcopy

#import numpy as np
from scipy.special import expit
from pyglmnet import utils


class GradientGroupLasso:
    
    def __init__(self, dg_M, df_M, reg_l1s, reg_l2, max_iter,learning_rate, tol, beta0_npm= None):
        
        n = dg_M.shape[0]
        d= dg_M.shape[1]
        m = df_M.shape[2]
        p = dg_M.shape[2]
        dummy_beta = np.ones((n,p,m))
        
        self.dg_M = dg_M
        self.df_M = df_M
        self.reg_l1s = reg_l1s
        self.reg_l2 = reg_l2
        self.beta0_npm = beta0_npm
        self.n = n
        self.p = p
        self.m = m 
        self.d = d
        self.dummy_beta = dummy_beta
        #self.group = np.asarray(group)
        self.max_iter = max_iter
        self.learning_rate = learning_rate
        self.tol = tol
        self.Tau = None
        self.alpha = 1.
        self.lossresults = {}
        self.dls = {}
        self.l2loss = {}
        self.penalty = {}
        
    def _prox(self,beta_npm, thresh):
        """Proximal operator."""
        
        p = self.p
        result = np.zeros(beta_npm.shape)
        result = np.asarray(result, dtype = float)
        for j in range(p):
            if np.linalg.norm(beta_npm[:,j,:]) > 0.:
                potentialoutput = beta_npm[:,j,:] - (thresh / np.linalg.norm(beta_npm[:,j,:])) * beta_npm[:,j,:]
                posind = np.asarray(np.where(beta_npm[:,j,:] > 0.))
                negind = np.asarray(np.where(beta_npm[:,j,:] < 0.))
                po = beta_npm[:,j,:].copy()
                po[posind[0],posind[1]] = np.asarray(np.clip(potentialoutput[posind[0],posind[1]],a_min = 0., a_max = 1e15), dtype = float)
                po[negind[0],negind[1]] = np.asarray(np.clip(potentialoutput[negind[0],negind[1]],a_min = -1e15, a_max = 0.), dtype = float)
                result[:,j,:] = po
        return result

    def _grad_L2loss(self, beta_npm):
        
        df_M = self.df_M
        dg_M = self.dg_M
        reg_l2 = self.reg_l2
        dummy_beta = self.dummy_beta
        
        df_M_hat = np.einsum('ndp,npm->ndm',dg_M, beta_npm)
        error = df_M_hat - df_M
        grad_beta = np.einsum('ndm,ndp->npm',error,dg_M) #+ reg_l2 * np.ones()
        #if 
        return grad_beta
    
    def _L1penalty(self, beta_npm):
        
        p = self.p
        m = self.m
        n = self.n 
        beta_mn_p = rearrange(beta_npm, 'n p m -> (m n) p')#np.reshape(beta_mnp, ((m*n,p)))
        L1penalty = np.linalg.norm(beta_mn_p, axis = 0).sum()
        
        return L1penalty
    
    def _loss(self,beta_npm, reg_lambda):
        """Define the objective function for elastic net."""
        L = self._logL(beta_npm)
        P = self._L1penalty(beta_npm)
        J = -L + reg_lambda * P
        return J
    
    def _logL(self,beta_npm):
        
        df_M = self.df_M
        dg_M = self.dg_M
        
        df_M_hat = np.einsum('ndp,npm -> ndm',dg_M, beta_npm)
        logL = -0.5 * np.linalg.norm((df_M - df_M_hat))**2
        return(logL)
    
    def _L2loss(self,beta_npm):
        output = -self._logL(beta_npm)
        return(output)

    def fhatlambda(self,learning_rate,beta_npm_new,beta_npm_old):

        print('lr',learning_rate)
        output = self._L2loss(beta_npm_old) + np.einsum('npm,npm', self._grad_L2loss(beta_npm_old),(beta_npm_new-beta_npm_old)) + (1/(2*learning_rate)) * np.linalg.norm(beta_npm_new-beta_npm_old)**2
        
        return(output)

    def _btalgorithm(self,beta_npm ,learning_rate,b,maxiter_bt,rl):
        
        grad_beta = self._grad_L2loss(beta_npm = beta_npm)
        for i in range(maxiter_bt):
            beta_npm_postgrad = beta_npm - learning_rate * grad_beta
            beta_npm_postgrad_postprox = self._prox(beta_npm_postgrad, learning_rate * rl)
            fz = self._L2loss(beta_npm_postgrad_postprox)
            #fhatz = self.fhatlambda(lam,beta_npm_postgrad_postprox, beta_npm_postgrad)
            fhatz = self.fhatlambda(learning_rate,beta_npm_postgrad_postprox, beta_npm)
            if fz <= fhatz:
                #print(i)
                break
            learning_rate = b*learning_rate    
            
        return(beta_npm_postgrad_postprox,learning_rate)
    
    def fit(self, beta0_npm = None):

        reg_l1s = self.reg_l1s
        n = self.n
        m = self.m
        p = self.p
        
        dg_M = self.dg_M
        df_M = self.df_M
        
        tol = self.tol
        np.random.RandomState(0)
        
        if beta0_npm is None:
            beta_npm_hat = 1 / (n*m*p) * np.random.normal(0.0, 1.0, [n, p,m])
            #1 / (n_features) * np.random.normal(0.0, 1.0, [n_features, n_classes])
        else: 
            beta_npm_hat = beta0_npm
            
        fit_params = list()
        for l, rl in enumerate(reg_l1s):
            fit_params.append({'beta': beta_npm_hat})
            if l == 0:
                fit_params[-1]['beta'] = beta_npm_hat
            else:
                fit_params[-1]['beta'] = fit_params[-2]['beta']
            
            alpha = 1.
            beta_npm_hat = fit_params[-1]['beta']
            #g = np.zeros([n_features, n_classes])
            L, DL ,L2,PEN = list(), list() , list(), list()
            learning_rate = self.learning_rate
            beta_npm_hat_1 = beta_npm_hat.copy()
            beta_npm_hat_2 = beta_npm_hat.copy()
            for t in range(0, self.max_iter):
                #print(t,l,rl)
                print(t)
                L.append(self._loss(beta_npm_hat, rl))
                L2.append(self._L2loss(beta_npm_hat))
                PEN.append(self._L1penalty(beta_npm_hat))
                w = (t / (t+ 3))
                beta_npm_hat_momentumguess = beta_npm_hat + w*(beta_npm_hat_1 - beta_npm_hat_2)
                
                beta_npm_hat , learning_rate = self._btalgorithm(beta_npm_hat_momentumguess,learning_rate,.5,1000, rl)
                #print(beta_npm_hat_momentumguess.max(), beta_npm_hat.max(),self._L2loss(beta_npm_hat), learning_rate)
                beta_npm_hat_2 = beta_npm_hat_1.copy()
                beta_npm_hat_1 = beta_npm_hat.copy()
                
                if t > 1:
                    DL.append(L[-1] - L[-2])
                    if np.abs(DL[-1] / L[-1]) < tol:
                        print('converged', rl)
                        msg = ('\tConverged. Loss function:'
                               ' {0:.2f}').format(L[-1])
                        msg = ('\tdL/L: {0:.6f}\n'.format(DL[-1] / L[-1]))
                        break

            fit_params[-1]['beta'] = beta_npm_hat
            self.lossresults[rl] = L
            self.l2loss[rl] = L2
            self.penalty[rl] = PEN
            self.dls[rl] = DL

        self.fit_ = fit_params
        #self.ynull_ = np.mean(y)

        return self

In [2]:
import matplotlib
matplotlib.use('Agg')
import os
import datetime
import numpy as np
import dill as pickle
import random
import sys
import seaborn as sns
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from matplotlib import rcParams
from collections import OrderedDict
import math
from matplotlib.lines import Line2D
from pylab import rcParams
from collections import Counter
from itertools import combinations
#from datetime import datetime

from shutil import copyfile
rcParams['figure.figsize'] = 25, 10

np.random.seed(0)
random.seed(0)
now = datetime.datetime.now().strftime("%B_%d_%Y_%H_%M_%S")
workingdirectory = os.popen('git rev-parse --show-toplevel').read()[:-1]
sys.path.append(workingdirectory)
os.chdir(workingdirectory)
from codes.experimentclasses.RigidEthanolPCA2 import RigidEthanolPCA2
from codes.otherfunctions.get_dictionaries import get_atoms_4
from codes.otherfunctions.get_grads import get_grads
from codes.otherfunctions.multirun import get_support_recovery_lambda
from codes.otherfunctions.multirun import get_lower_interesting_lambda
from codes.otherfunctions.multirun import get_coeffs_and_lambdas
from codes.otherfunctions.multirun import get_support
from codes.otherfunctions.multiplot import plot_support_2d
from codes.otherfunctions.multiplot import plot_reg_path_ax_lambdasearch
from codes.otherfunctions.multiplot import plot_gs_v_dgnorm
from codes.otherfunctions.multiplot import plot_dot_distributions
from codes.otherfunctions.multirun import get_cosines
from codes.flasso.Replicate import Replicate
from codes.otherfunctions.multirun import get_olsnorm_and_supportsbrute
from codes.otherfunctions.multiplot import highlight_cell

from codes.geometer.RiemannianManifold import RiemannianManifold
from codes.geometer.ShapeSpace import ShapeSpace
from codes.geometer.TangentBundle import TangentBundle

def plot_watch_custom(to_plot, p, ax,colors):
    #fig, ax = plt.subplots(figsize = (15,15))
    #%matplotlib inline
    
    #fig, ax = plt.subplots(figsize = (15,15))
    theta = np.linspace(0, 2*np.pi, 10000)
    cmap = plt.get_cmap('twilight_shifted',p)
    
    angles = np.linspace(0, 2*np.pi, p+1)
    
    radius = 1.

    a = radius*np.cos(theta)
    b = radius*np.sin(theta)

    #figure, axes = plt.subplots(figsize = (15,15))

    #axes.plot(a, b, color= 'gray')
    ax.scatter(a, b, color = 'gray', s= .2, alpha = .1)#, '-', color = 'gray')#, s= .1, alpha = .1)#, type = 'line')#,cmap=plt.get_cmap('twilight')) #'hsv','twilight_shifted

    #for i in range(to_plot.shape)
    if len(to_plot.shape) > 1:
        totes = np.sum(to_plot, axis = 0)
    else:
        totes = to_plot
        
    for j in range(p):
        print(np.cos(angles[j]), np.sin(angles[j]))#r'$test \frac{1}{}$'.format(g)
        ax.scatter(np.cos(angles[j]),np.sin(angles[j]),color=cmap.colors[j], marker  = 'x')
        ax.text( x = 1.1*np.cos(angles[j]),
                  y = 1.1*np.sin(angles[j]),
                  s = r"$g_{{{}}}$".format(j),color=colors[j],#cmap.colors[j],
                  fontdict = {'fontsize' : 70},
                  horizontalalignment='center',
         verticalalignment='center')



        ax.text( x = .9*np.cos(angles[j]),y = .9*np.sin(angles[j]),s = str(totes[j] / nreps), fontdict = {'fontsize' : 40},
                  horizontalalignment='center',
         verticalalignment='center')



    for j in range(p):
        ax.scatter(np.cos(angles[j]),np.sin(angles[j]),color=colors[j], marker  = 'o', s = 200*totes[j] )

    if len(to_plot.shape) > 1:
        for i in range(p):
            for j in range(p):

                #point1 = [1, 2]
                #point2 = [3, 4]

                x_values = [np.cos(angles[j]), np.cos(angles[i])]
                #gather x-values

                y_values = [np.sin(angles[j]), np.sin(angles[i])]
                #gather y-values

                ax.plot(x_values, y_values,linewidth = to_plot[i,j], color = 'black')

                if to_plot[i,j] > 0 :
                    ax.text( x = np.mean(x_values),
                      y = np.mean(y_values),
                      s = str(to_plot[i,j] / nreps),
                      fontdict = {'fontsize' : 40})#,
                  #horizontalalignment='left',
                # verticalalignment='bottom')

                #axes.axline((x1, y1), (x2, y2))
    ax.set_aspect(1)
    ax.set_axis_off()
    ax.set_title(r"$\omega = 25$")
    
def plot_cosines(cosines, ax, colors):
    p = cosines.shape[0]
    sns.heatmap(cosines ,ax = ax, vmin = 0., vmax = 1.)
#    ax = sns.heatmap(x, cmap=cmap)
    # use matplotlib.colorbar.Colorbar object
    cbar = ax.collections[0].colorbar
    # here set the labelsize by 20
    cbar.ax.tick_params(labelsize=20)

    for xtick, color in zip(ax.get_xticklabels(), colors):
        xtick.set_color(color)
    for ytick, color in zip(ax.get_yticklabels(), colors):
        ytick.set_color(color)
    ax.set_xticklabels(ax.get_xmajorticklabels(), fontsize = 500 / p)
    ax.set_yticklabels(ax.get_ymajorticklabels(), fontsize = 500 / p)
    
    ax.set_ylabel(r"$g_{j'}$", fontsize = 70)
    ax.set_xlabel(r"$g_{j}$", fontsize = 70)
    #ax.set_title(r"$\text{hi}$")
    ax.set_title(r"$\frac{1}{n'} \sum_{i = 1}^{n'} \frac{\langle grad_{\mathcal M} g_j (\xi_i) ,grad_{\mathcal M} g_{j'} (\xi_i)\rangle}{\|grad_{\mathcal M} g_j (\xi_i) \|_2 \| grad_{\mathcal M} g_{j'}(\xi_i) \|_2} $",
                fontsize = 70)

# def plot_reg_path_ax_lambdasearch_customcolors_norm(ax, coeffs, xaxis,fig, colors):
#     p = coeffs.shape[3]
#     q = coeffs.shape[1]
#     gnames = np.asarray(list(range(p)), dtype=str)

#     rcParams['axes.titlesize'] = 30
#     plt.rc('text', usetex=True)

#     normax = np.sqrt(np.sum(np.sum(np.sum(coeffs ** 2, axis=1), axis=1), axis=1).max())

#     for j in range(p):
#         toplot = np.linalg.norm(np.linalg.norm(coeffs[:, :, :, j], axis=2), axis=1)
#         # axes[0].boxplot(toplot, positions=xaxis, showfliers=False, vert=True, widths=widths,medianprops=dict(linestyle=''))
#         ax.plot(xaxis, toplot, 'go--', linewidth=5, markersize=0, alpha=1.,
#                      color=colors[j], label=gnames[j])

#     kkk = xaxis.copy()
#     kkk.sort()

#     # xupperindex = np.min(np.where(np.sum(np.sum(np.sum(coeffs**2, axis = 1), axis = 1), axis = 1) ==0)[0])

#     #for k in range(1 + q):
#     ax.tick_params(labelsize=50)
#     ax.set_xscale('symlog')
#     ax.set_yscale('symlog')
#     ax.set_ylim(bottom=0, top=normax)
#     # axes[k].set_xlim(left = 0, right = xaxis[xupperindex])
#     #if (k == 0):
#     tixx = np.hstack(
#         [np.asarray([0]), 10 ** np.linspace(math.floor(np.log10(normax)), math.floor(np.log10(normax)) + 1, 2)])
# #    if k != 0:
#         # axes[k].set_yticks(tixx)
#     ax.set_ylabel(r"$\displaystyle \|\hat \beta_{j}\|_2$", fontsize = 70)
#     ax.set_xlabel(r"$\lambda  $", fontsize = 70)#\sqrt{nm}
#     #ylabel = r"$\displaystyle \|\hat \beta_{j}\|_2$"
#     #ax.l
#     #if k == 0:
#     #ax.set_title("Combined", fontdict={'fontsize': 50})
#     ax.grid(True, which="both", alpha=True)

#     #handles, labels = ax.get_legend_handles_labels()
#     #by_label = OrderedDict(zip(labels, handles))
#     # fig.text(0.5, 0.04, xlabel, ha='center', va='center', fontsize=50)
#     # fig.text(0.05, 0.5, ylabel, ha='center', va='center', rotation='vertical', fontsize=60)
#     #fig.subplots_adjust(right=0.75)
#     #leg_ax = fig.add_axes([.8, 0.15, 0.05, 0.7])
#     #leg_ax.axis('off')
#     #leg = leg_ax.legend(by_label.values(), gnames, prop={'size': 200 / p})
#     # leg.set_title('Torsion', prop={'size': Function})
#     #for l in leg.get_lines():
#     #    l.set_alpha(1)
#     # fig.savefig(filename + 'beta_paths_n' + str(n) + 'nsel' + str(nsel) + 'nreps' + str(
#     #    nreps))
    

def plot_reg_path_ax_lambdasearch_customcolors(axes, coeffs, xaxis,fig, colors):
    p = coeffs.shape[3]
    q = coeffs.shape[1]
    gnames = np.asarray(list(range(p)), dtype=str)

    # xlabel = r"$\displaystyle \lambda$"
    # ylabel = r"$\displaystyle \|\hat \beta_{j}\|_2$"
    rcParams['axes.titlesize'] = 30
    plt.rc('text', usetex=True)

    # maxes = np.zeros(q)
    # for k in range(q):
    #     maxes[k] = np.linalg.norm(coeffs[:, k, :, :], axis=1).max()
    # normax = maxes.max()
    normax = np.sqrt(np.sum(np.sum(np.sum(coeffs ** 2, axis=1), axis=1), axis=1).max())

    for k in range(q):
        for j in range(p):
            toplot = np.linalg.norm(coeffs[:, k, :, j], axis=1)
            w = .15
            widths = np.asarray([width(xaxis[i], w) for i in range(len(xaxis))])
            # axes[k+1].boxplot(toplot, positions=xaxis, showfliers=False, vert=True, widths=widths,medianprops=dict(linestyle=''))
            axes[k + 1].plot(xaxis, toplot, 'go--', linewidth=5, markersize=0, alpha=1.,
                             color=colors[j], label=gnames[j])
    for j in range(p):
        toplot = np.linalg.norm(np.linalg.norm(coeffs[:, :, :, j], axis=2), axis=1)
        # axes[0].boxplot(toplot, positions=xaxis, showfliers=False, vert=True, widths=widths,medianprops=dict(linestyle=''))
        axes[0].plot(xaxis, toplot, 'go--', linewidth=5, markersize=0, alpha=1.,
                     color=colors[j], label=gnames[j])

    kkk = xaxis.copy()
    kkk.sort()

    # xupperindex = np.min(np.where(np.sum(np.sum(np.sum(coeffs**2, axis = 1), axis = 1), axis = 1) ==0)[0])

    for k in range(1 + q):
        axes[k].tick_params(labelsize=50)
        axes[k].set_xscale('symlog')
        axes[k].set_yscale('symlog')
        axes[k].set_ylim(bottom=0, top=normax)
        # axes[k].set_xlim(left = 0, right = xaxis[xupperindex])
        if (k == 0):
            tixx = np.hstack(
                [np.asarray([0]), 10 ** np.linspace(math.floor(np.log10(normax)), math.floor(np.log10(normax)) + 1, 2)])
        if k != 0:
            # axes[k].set_yticks(tixx)
            axes[k].set_yticklabels([])
        if k != q:
            axes[k+1].set_title(r"$\phi_{{{}}}$".format(k+1), fontsize = 50)
            #axes[k + 1].set_title(r"$\phi_{{{}}}$.format(k)")
        if k == 0:
            axes[k].set_title("Combined", fontdict={'fontsize': 50})
    for k in range(1 + q):
        axes[k].grid(True, which="both", alpha=True)
        axes[k].set_xlabel(r"$\lambda$", fontsize = 50)
        
    axes[0].set_ylabel(r"$\|\beta\|$", fontsize = 50)
        
    handles, labels = axes[0].get_legend_handles_labels()
    by_label = OrderedDict(zip(labels, handles))
    # fig.text(0.5, 0.04, xlabel, ha='center', va='center', fontsize=50)
    # fig.text(0.05, 0.5, ylabel, ha='center', va='center', rotation='vertical', fontsize=60)
    fig.subplots_adjust(right=0.75)
    leg_ax = fig.add_axes([.8, 0.15, 0.05, 0.7])
    leg_ax.axis('off')
    leg = leg_ax.legend(by_label.values(), gnames, prop={'size': 200 / p})
    # leg.set_title('Torsion', prop={'size': Function})
    for l in leg.get_lines():
        l.set_alpha(1)
    # fig.savefig(filename + 'beta_paths_n' + str(n) + 'nsel' + str(nsel) + 'nreps' + str(
    #    nreps))


    
def get_grads(experiment, Mpca, Mangles, N, selected_points):
    dimnoise = experiment.dimnoise
    dim = experiment.dim
    cores = experiment.cores

    tangent_bases = Mpca.get_wlpca_tangent_sel(Mpca, selected_points, dimnoise)
    subM = RiemannianManifold(Mpca.data[selected_points], dim)
    subM.tb = TangentBundle(subM, tangent_bases)
    N.tangent_bundle = TangentBundle(N, np.swapaxes(N.geom.rmetric.Hvv[:,:dim,:],1,2))

    df_M = experiment.get_dF_js_idM(Mpca, N, subM.tb, N.tangent_bundle, selected_points, dimnoise)
    df_M2 = df_M / np.sum(np.linalg.norm(df_M, axis=1) ** 2, axis=0)**(0.5)
    dg_x = experiment.get_dx_g_full(Mangles.data[selected_points])

    W = ShapeSpace(experiment.positions, Mangles.data)
    dw = W.get_dw(cores, experiment.atoms3, experiment.natoms, selected_points)
    dg_w = experiment.project(np.swapaxes(dw, 1, 2),
                              experiment.project(dw, dg_x))

    dg_w_pca = np.asarray([np.matmul(experiment.projector, dg_w[j].transpose()).transpose() for j in range(len(selected_points))])
    dgw_norm = experiment.normalize(dg_w_pca)
    dg_M = experiment.project(subM.tb.tangent_bases, dgw_norm)
    return (df_M2, dg_M, dg_w, dg_w_pca, dgw_norm)

#set parameters
n = 10000 #number of data points to simulate
nsel = 100 #number of points to analyze with lasso
tol = 1e-14 #convergence criteria for lasso
n_neighbors = 1000 #number of neighbors in megaman
m = 3 #number of embedding dimensions (diffusion maps)
diffusion_time = 0.05 #(yuchia suggestion)
dim = 2 #manifold dimension
dimnoise = 2
natoms = 9
cores = 3 #number of cores for parallel processing
cor = 0.0 #correlation for noise
var = 0.0001 #variance scaler for noise
cor = 0.0 #correlation for noise
var = 10. #variance scaler for noise
ii = np.asarray([0,0,0,0,1,1,1,2]) # atom adjacencies for dihedral angle computation
jj = np.asarray([1,2,3,4,5,6,7,8])

#run experiment
atoms4 = np.asarray([[6,1,0,4],[4,0,2,8],[7,6,5,1],[3,0,2,4]],dtype = int)
nreps = 25
lambda_max = 1
max_search = 30

folder = workingdirectory + '/Figures/rigidethanol/' + now + 'n' + str(n) + 'nsel' + str(nsel) + 'nreps' + str(nreps)
os.mkdir(folder)


/Users/samsonkoelle/manifoldflasso_jmlr


In [3]:
from codes.experimentclasses.AtomicRegression import AtomicRegression
from codes.geometer.RiemannianManifold import RiemannianManifold
from codes.otherfunctions.data_stream import data_stream
from codes.geometer.ShapeSpace import compute3angles
import numpy as np
import math
from sklearn.decomposition import TruncatedSVD
from pathos.multiprocessing import ProcessingPool as Pool
#this is for use with full dictionaries
#all points are members of a different irrational field
#in order to avoid set of measure zero
#in the noiseless case
from codes.experimentclasses.AtomicRegression import AtomicRegression
from codes.geometer.RiemannianManifold import RiemannianManifold
from codes.otherfunctions.data_stream import data_stream
from codes.geometer.ShapeSpace import compute3angles
import numpy as np
import math
from sklearn.decomposition import TruncatedSVD
from pathos.multiprocessing import ProcessingPool as Pool

class RigidEthanolPCA2(AtomicRegression):
    """
    Parameters
    ----------
    cor : string,
        Data file to load
    xvar : np.array(dtype = int),
        List of adjacencies
    jj : np.array,
        List of adjacencies part 2
    d : int,
        dimension over which to evaluate the radii (smaller usually better)
    rmin : float,
        smallest radius ( = rad_bw_ratio * bandwidth) to consider
    rmax : float,
        largest radius ( = rad_bw_ratio * bandwidth) to consider
    ntry : int,
        number of radii between rmax and rmin to try
    run_parallel : bool,
        whether to run the analysis in parallel over radii
    search_space : str,
        either 'linspace' or 'logspace', choose to search in log or linear space
    rad_bw_ratio : float,
        the ratio of radius and kernel bandwidth, default to be 3 (radius = 3*h)
    Methods
    -------
    generate_data :
        Simulates data
    get_atoms_4 :
    	Gets atomic tetrahedra based off of ii and jj
    get_atoms_3 :
    	Gets triples of atoms

    """

    # AtomicRegression(dim, ii, jj, filename)
    def __init__(self, dim, cor, xvar,ii,jj, cores, noise, custom_bonds = None):
        natoms = 9
        n = 10000
        self.cor = cor
        self.xvar = xvar
        self.cores = cores
        self.noise = noise
        AtomicRegression.__init__(self, dim,n,ii,jj, natoms, cores)
        if custom_bonds.any() != None:
            self.atoms4 = custom_bonds
            self.p = custom_bonds.shape[0]

    def generate_data(self, noise=False):
        
        
        n = self.n
        d = self.d
        cor = self.cor
        xvar = self.xvar
        natoms = self.natoms
        cores = self.cores
        atoms3 = self.atoms3
        dim = self.dim
        #noise = self.noise

        positions = np.zeros((n, 9, 3))
        # positions[0,0,:] = np.asarray([0.,0.,0.])
        # positions[0,1,:] = np.asarray([-10.,0.,0.])
        # positions[0,2,:] = np.asarray([1.,10.,0.])
        # #positions[0,8,:] = np.asarray([1.,1.,0.])
        # positions[0,8,:] = np.asarray([2.,10.,-0])
        # positions[0,3,:] = np.asarray([1., np.cos(2/3 * np.pi), np.sin(2/3 * np.pi)])
        # positions[0,4,:] = np.asarray([1., np.cos(2/3 * np.pi), np.sin(4/3 * np.pi)])
        # positions[0,5,:] = np.asarray([-12.,1.,0.])
        # positions[0,6,:] = np.asarray([-12., np.cos(2/3 * np.pi),np.sin(2/3 * np.pi)])
        # positions[0,7,:] = np.asarray([-12.,np.cos(2/3 * np.pi), np.sin(4/3 * np.pi)])
        positions[0,0,:] = np.asarray([0.,0.,0.])
        positions[0,1,:] = np.asarray([-10.,0.,np.sqrt(2)/100])
        positions[0,2,:] = np.asarray([0.,10.,np.sqrt(3)/100])
        #positions[0,8,:] = np.asarray([1.,1.,0.])
        positions[0,8,:] = np.asarray([1.,10.,np.sqrt(5)/100])
        positions[0,3,:] = np.asarray([np.sqrt(7)/100, np.cos(2/3 * np.pi), np.sin(2/3 * np.pi)])
        positions[0,4,:] = np.asarray([np.sqrt(11)/100, np.cos(2/3 * np.pi), np.sin(4/3 * np.pi)])
        positions[0,5,:] = np.asarray([-11.,1.,np.sqrt(13)/100])
        positions[0,6,:] = np.asarray([-11. , np.cos(2/3 * np.pi)+ np.sqrt(17)/1000,np.sin(2/3 * np.pi) ])
        positions[0,7,:] = np.asarray([-11., np.cos(2/3 * np.pi)+ np.sqrt(19)/100, np.sin(4/3 * np.pi)])

        angles1 = np.tile(np.linspace(start=0., stop=2 * math.pi, num=int(np.sqrt(n)), endpoint=False),
                          int(np.sqrt(n)))
        angles2 = np.repeat(np.linspace(start=0., stop=2 * math.pi, num=int(np.sqrt(n)), endpoint=False),
                            int(np.sqrt(n)))
        for i in range(1, n):
            rotationmatrix1 = np.zeros((3, 3))
            rotationmatrix1[1, 1] = 1
            rotationmatrix1[0, 0] = np.cos(angles1[i])
            rotationmatrix1[0, 2] = -np.sin(angles1[i])
            rotationmatrix1[2, 2] = np.cos(angles1[i])
            rotationmatrix1[2, 0] = np.sin(angles1[i])
            rotationmatrix2 = np.zeros((3, 3))
            rotationmatrix2[0, 0] = 1
            rotationmatrix2[1, 1] = np.cos(angles2[i])
            rotationmatrix2[1, 2] = -np.sin(angles2[i])
            rotationmatrix2[2, 2] = np.cos(angles2[i])
            rotationmatrix2[2, 1] = np.sin(angles2[i])
            positions[i, np.asarray([3, 4]), :] = positions[0, np.asarray([3, 4]), :]
            positions[i, np.asarray([2, 8]), :] = np.matmul(rotationmatrix1,
                                                            positions[0, np.asarray([2, 8]),
                                                            :].transpose()).transpose()
            positions[i, np.asarray([1, 5, 6, 7]), :] = np.matmul(rotationmatrix2,
                                                                  positions[0, np.asarray([1, 5, 6, 7]),
                                                                  :].transpose()).transpose()

        covariance = np.identity(natoms)
        for i in range(natoms):
            for j in range(natoms):
                if i != j:
                    covariance[i, j] = cor
        covariance = xvar * covariance
        print(covariance)
        if noise == True:
            for i in range(n):
                for j in range(3):
                    
                    positions[i, :, j] = np.random.multivariate_normal(positions[i, :, j], covariance, size=1)
        self.positions = positions
        p = Pool(cores)
        results = p.map(lambda i: compute3angles(position=positions[i[0], atoms3[i[1]], :]),
                            data_stream(10000, 84))
        data = np.reshape(results, (n, (d)))
        svd = TruncatedSVD(n_components=50)
        svd.fit(data)
        data_pca = svd.transform(data)
        return (RiemannianManifold(data, dim), RiemannianManifold(data_pca,dim), svd.components_)
    
def plot_watch_custom(to_plot, p, ax,colors):
    #fig, ax = plt.subplots(figsize = (15,15))
    #%matplotlib inline
    
    #fig, ax = plt.subplots(figsize = (15,15))
    theta = np.linspace(0, 2*np.pi, 10000)
    cmap = plt.get_cmap('twilight_shifted',p)
    
    angles = np.linspace(0, 2*np.pi, p+1)
    
    radius = 1.

    a = radius*np.cos(theta)
    b = radius*np.sin(theta)

    #figure, axes = plt.subplots(figsize = (15,15))

    #axes.plot(a, b, color= 'gray')
    ax.scatter(a, b, color = 'gray', s= .2, alpha = .1)#, '-', color = 'gray')#, s= .1, alpha = .1)#, type = 'line')#,cmap=plt.get_cmap('twilight')) #'hsv','twilight_shifted

    #for i in range(to_plot.shape)
    if len(to_plot.shape) > 1:
        totes = np.sum(to_plot, axis = 0)
    else:
        totes = to_plot
        
    for j in range(p):
        print(np.cos(angles[j]), np.sin(angles[j]))#r'$test \frac{1}{}$'.format(g)
        ax.scatter(np.cos(angles[j]),np.sin(angles[j]),color=cmap.colors[j], marker  = 'x')
        ax.text( x = 1.1*np.cos(angles[j]),
                  y = 1.1*np.sin(angles[j]),
                  s = r"$g_{{{}}}$".format(j),color=colors[j],#cmap.colors[j],
                  fontdict = {'fontsize' : 70},
                  horizontalalignment='center',
         verticalalignment='center')



        ax.text( x = .9*np.cos(angles[j]),y = .9*np.sin(angles[j]),s = str(totes[j] / nreps), fontdict = {'fontsize' : 40},
                  horizontalalignment='center',
         verticalalignment='center')



    for j in range(p):
        ax.scatter(np.cos(angles[j]),np.sin(angles[j]),color=colors[j], marker  = 'o', s = 200*totes[j] )

    if len(to_plot.shape) > 1:
        for i in range(p):
            for j in range(p):

                #point1 = [1, 2]
                #point2 = [3, 4]

                x_values = [np.cos(angles[j]), np.cos(angles[i])]
                #gather x-values

                y_values = [np.sin(angles[j]), np.sin(angles[i])]
                #gather y-values

                ax.plot(x_values, y_values,linewidth = to_plot[i,j], color = 'black')

                if to_plot[i,j] > 0 :
                    ax.text( x = np.mean(x_values),
                      y = np.mean(y_values),
                      s = str(to_plot[i,j] / nreps),
                      fontdict = {'fontsize' : 40})#,
                  #horizontalalignment='left',
                # verticalalignment='bottom')

                #axes.axline((x1, y1), (x2, y2))
    ax.set_aspect(1)
    ax.set_axis_off()
    ax.set_title(r"$\omega = 25$")
    
def plot_cosines(cosines, ax, colors):
    p = cosines.shape[0]
    sns.heatmap(cosines ,ax = ax, vmin = 0., vmax = 1.)
#    ax = sns.heatmap(x, cmap=cmap)
    # use matplotlib.colorbar.Colorbar object
    cbar = ax.collections[0].colorbar
    # here set the labelsize by 20
    cbar.ax.tick_params(labelsize=20)

    for xtick, color in zip(ax.get_xticklabels(), colors):
        xtick.set_color(color)
    for ytick, color in zip(ax.get_yticklabels(), colors):
        ytick.set_color(color)
    ax.set_xticklabels(ax.get_xmajorticklabels(), fontsize = 500 / p)
    ax.set_yticklabels(ax.get_ymajorticklabels(), fontsize = 500 / p)
    
    ax.set_ylabel(r"$g_{j'}$", fontsize = 70)
    ax.set_xlabel(r"$g_{j}$", fontsize = 70)
    #ax.set_title(r"$\text{hi}$")
    ax.set_title(r"$\frac{1}{n'} \sum_{i = 1}^{n'} \frac{\langle grad_{\mathcal M} g_j (\xi_i) ,grad_{\mathcal M} g_{j'} (\xi_i)\rangle}{\|grad_{\mathcal M} g_j (\xi_i) \|_2 \| grad_{\mathcal M} g_{j'}(\xi_i) \|_2} $",
                fontsize = 70)

# def plot_reg_path_ax_lambdasearch_customcolors_norm(ax, coeffs, xaxis,fig, colors):
#     p = coeffs.shape[3]
#     q = coeffs.shape[1]
#     gnames = np.asarray(list(range(p)), dtype=str)

#     rcParams['axes.titlesize'] = 30
#     plt.rc('text', usetex=True)

#     normax = np.sqrt(np.sum(np.sum(np.sum(coeffs ** 2, axis=1), axis=1), axis=1).max())

#     for j in range(p):
#         toplot = np.linalg.norm(np.linalg.norm(coeffs[:, :, :, j], axis=2), axis=1)
#         # axes[0].boxplot(toplot, positions=xaxis, showfliers=False, vert=True, widths=widths,medianprops=dict(linestyle=''))
#         ax.plot(xaxis, toplot, 'go--', linewidth=5, markersize=0, alpha=1.,
#                      color=colors[j], label=gnames[j])

#     kkk = xaxis.copy()
#     kkk.sort()

#     # xupperindex = np.min(np.where(np.sum(np.sum(np.sum(coeffs**2, axis = 1), axis = 1), axis = 1) ==0)[0])

#     #for k in range(1 + q):
#     ax.tick_params(labelsize=50)
#     ax.set_xscale('symlog')
#     ax.set_yscale('symlog')
#     ax.set_ylim(bottom=0, top=normax)
#     # axes[k].set_xlim(left = 0, right = xaxis[xupperindex])
#     #if (k == 0):
#     tixx = np.hstack(
#         [np.asarray([0]), 10 ** np.linspace(math.floor(np.log10(normax)), math.floor(np.log10(normax)) + 1, 2)])
# #    if k != 0:
#         # axes[k].set_yticks(tixx)
#     ax.set_ylabel(r"$\displaystyle \|\hat \beta_{j}\|_2$", fontsize = 70)
#     ax.set_xlabel(r"$\lambda  $", fontsize = 70)#\sqrt{nm}
#     #ylabel = r"$\displaystyle \|\hat \beta_{j}\|_2$"
#     #ax.l
#     #if k == 0:
#     #ax.set_title("Combined", fontdict={'fontsize': 50})
#     ax.grid(True, which="both", alpha=True)

#     #handles, labels = ax.get_legend_handles_labels()
#     #by_label = OrderedDict(zip(labels, handles))
#     # fig.text(0.5, 0.04, xlabel, ha='center', va='center', fontsize=50)
#     # fig.text(0.05, 0.5, ylabel, ha='center', va='center', rotation='vertical', fontsize=60)
#     #fig.subplots_adjust(right=0.75)
#     #leg_ax = fig.add_axes([.8, 0.15, 0.05, 0.7])
#     #leg_ax.axis('off')
#     #leg = leg_ax.legend(by_label.values(), gnames, prop={'size': 200 / p})
#     # leg.set_title('Torsion', prop={'size': Function})
#     #for l in leg.get_lines():
#     #    l.set_alpha(1)
#     # fig.savefig(filename + 'beta_paths_n' + str(n) + 'nsel' + str(nsel) + 'nreps' + str(
#     #    nreps))
    

def plot_reg_path_ax_lambdasearch_customcolors(axes, coeffs, xaxis,fig, colors):
    p = coeffs.shape[3]
    q = coeffs.shape[1]
    gnames = np.asarray(list(range(p)), dtype=str)

    # xlabel = r"$\displaystyle \lambda$"
    # ylabel = r"$\displaystyle \|\hat \beta_{j}\|_2$"
    rcParams['axes.titlesize'] = 30
    plt.rc('text', usetex=True)

    # maxes = np.zeros(q)
    # for k in range(q):
    #     maxes[k] = np.linalg.norm(coeffs[:, k, :, :], axis=1).max()
    # normax = maxes.max()
    normax = np.sqrt(np.sum(np.sum(np.sum(coeffs ** 2, axis=1), axis=1), axis=1).max())

    for k in range(q):
        for j in range(p):
            toplot = np.linalg.norm(coeffs[:, k, :, j], axis=1)
            w = .15
            widths = np.asarray([width(xaxis[i], w) for i in range(len(xaxis))])
            # axes[k+1].boxplot(toplot, positions=xaxis, showfliers=False, vert=True, widths=widths,medianprops=dict(linestyle=''))
            axes[k + 1].plot(xaxis, toplot, 'go--', linewidth=5, markersize=0, alpha=1.,
                             color=colors[j], label=gnames[j])
    for j in range(p):
        toplot = np.linalg.norm(np.linalg.norm(coeffs[:, :, :, j], axis=2), axis=1)
        # axes[0].boxplot(toplot, positions=xaxis, showfliers=False, vert=True, widths=widths,medianprops=dict(linestyle=''))
        axes[0].plot(xaxis, toplot, 'go--', linewidth=5, markersize=0, alpha=1.,
                     color=colors[j], label=gnames[j])

    kkk = xaxis.copy()
    kkk.sort()

    # xupperindex = np.min(np.where(np.sum(np.sum(np.sum(coeffs**2, axis = 1), axis = 1), axis = 1) ==0)[0])

    for k in range(1 + q):
        axes[k].tick_params(labelsize=50)
        axes[k].set_xscale('symlog')
        axes[k].set_yscale('symlog')
        axes[k].set_ylim(bottom=0, top=normax)
        # axes[k].set_xlim(left = 0, right = xaxis[xupperindex])
        if (k == 0):
            tixx = np.hstack(
                [np.asarray([0]), 10 ** np.linspace(math.floor(np.log10(normax)), math.floor(np.log10(normax)) + 1, 2)])
        if k != 0:
            # axes[k].set_yticks(tixx)
            axes[k].set_yticklabels([])
        if k != q:
            axes[k+1].set_title(r"$\phi_{{{}}}$".format(k+1), fontsize = 50)
            #axes[k + 1].set_title(r"$\phi_{{{}}}$.format(k)")
        if k == 0:
            axes[k].set_title("Combined", fontdict={'fontsize': 50})
    for k in range(1 + q):
        axes[k].grid(True, which="both", alpha=True)
        axes[k].set_xlabel(r"$\lambda$", fontsize = 50)
        
    axes[0].set_ylabel(r"$\|\beta\|$", fontsize = 50)
        
    handles, labels = axes[0].get_legend_handles_labels()
    by_label = OrderedDict(zip(labels, handles))
    # fig.text(0.5, 0.04, xlabel, ha='center', va='center', fontsize=50)
    # fig.text(0.05, 0.5, ylabel, ha='center', va='center', rotation='vertical', fontsize=60)
    fig.subplots_adjust(right=0.75)
    leg_ax = fig.add_axes([.8, 0.15, 0.05, 0.7])
    leg_ax.axis('off')
    leg = leg_ax.legend(by_label.values(), gnames, prop={'size': 200 / p})
    # leg.set_title('Torsion', prop={'size': Function})
    for l in leg.get_lines():
        l.set_alpha(1)
    # fig.savefig(filename + 'beta_paths_n' + str(n) + 'nsel' + str(nsel) + 'nreps' + str(
    #    nreps))



In [4]:
new_MN = True
new_grad = True
savename = 'rigidethanol_jmlrresub'
savefolder = 'rigidethanol'
loadfolder = 'rigidethanol'
loadname = 'rigidethanol_jmlrresub'
var = 0.000001#var = 0.01 breaks
if new_MN == True:
    experiment = RigidEthanolPCA2(dim, cor, var, ii, jj, cores, True, atoms4)
    experiment.M, experiment.Mpca, projector = experiment.generate_data(noise=True)
    experiment.q = m
    experiment.m = m
    experiment.dimnoise = dimnoise
    experiment.projector = projector
    experiment.Mpca.geom = experiment.Mpca.compute_geom(diffusion_time, n_neighbors)
    experiment.N = experiment.Mpca.get_embedding3(experiment.Mpca.geom, m, diffusion_time, dim)
    # with open(workingdirectory + '/untracked_data/embeddings/' + savefolder + '/' + savename + '.pkl' ,
    #          'wb') as output:
    #      pickle.dump(experiment, output, pickle.HIGHEST_PROTOCOL)

atoms4,p = get_atoms_4(natoms,ii,jj)
experiment.p = p
experiment.atoms4 = atoms4
#experiment.itermax = itermax
experiment.tol = tol
experiment.dnoise = dim
experiment.nreps = nreps
experiment.nsel = nsel
experiment.folder = folder

replicates = {}
selected_points_save = np.zeros((nreps,nsel))



[[1.e-06 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00]
 [0.e+00 1.e-06 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00]
 [0.e+00 0.e+00 1.e-06 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00]
 [0.e+00 0.e+00 0.e+00 1.e-06 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00]
 [0.e+00 0.e+00 0.e+00 0.e+00 1.e-06 0.e+00 0.e+00 0.e+00 0.e+00]
 [0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 1.e-06 0.e+00 0.e+00 0.e+00]
 [0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 1.e-06 0.e+00 0.e+00]
 [0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 1.e-06 0.e+00]
 [0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 1.e-06]]


In [5]:
print('pre-gradient acquisition')
print(datetime.datetime.now())
for i in range(1):#nreps):
    print(i)
    selected_points = np.random.choice(list(range(n)),nsel,replace = False)
    selected_points_save[i] = selected_points
    replicates[i] = Replicate()
    replicates[i].nsel = nsel
    replicates[i].selected_points = selected_points
    replicates[i].df_M,replicates[i].dg_M,replicates[i].dg_w ,replicates[i].dg_w_pca ,replicates[i].dgw_norm  = get_grads(experiment, experiment.Mpca, experiment.M, experiment.N, selected_points)
    replicates[i].dg_M = np.swapaxes(replicates[i].dg_M, 1,2)

pre-gradient acquisition
2021-02-05 11:31:52.879091
0


In [6]:
savename = 'rigidethanol_010521_p12rep5n500_papernorm_noise_p0001'
savefolder = 'rigidethanol'
loadfolder = 'rigidethanol'
loadname = 'rigidethanol_010521_p12rep5n500_papernorm_noise_p0001'

# with open(workingdirectory + '/untracked_data/embeddings/' + savefolder + '/' + savename + 'replicates.pkl' ,
#          'wb') as output:
#      pickle.dump(replicates, output, pickle.HIGHEST_PROTOCOL)


selected_points_save = np.asarray(selected_points_save, dtype = int)
gl_itermax = 500
lambdas_start = [0.,.0005 * np.sqrt(nsel * p)]
max_search = 30
reg_l2 = 0.
card = dim
tol = 1e-14
learning_rate = 100

from pathos.multiprocessing import ProcessingPool as Pool
#from codes.flasso.GradientGroupLasso import batch_stream, get_sr_lambda_sam_parallel

print('pre-gradient descent')
print(datetime.datetime.now())
cores = 16
# pcor = Pool(cores)
# results = pcor.map(lambda replicate: get_sr_lambda_sam_parallel(replicate, gl_itermax, lambdas_start,reg_l2, max_search, card, tol,learning_rate), batch_stream(replicates))

results = {}
for r in range(nreps):#nreps):
    print(r)
    results[r] = Replicate()
    #results[r] = get_sr_lambda_sam_parallel(replicates[r], gl_itermax, lambdas_start,reg_l2, max_search, card, tol,learning_rate)


pre-gradient descent
2021-02-05 11:32:11.579705
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24


In [11]:
r = 0 
np.linalg.norm(np.einsum('n d m, n d p -> n p m ' ,replicates[r].df_M , replicates[r].dg_M), axis = tuple([0,2])).max()

7.5731865294897025

In [14]:
#max over n
r = 0 
GGL = GradientGroupLasso(replicates[r].dg_M, replicates[r].df_M, np.asarray([7.58]), np.asarray([0.]), 500,.5, 1e-14, beta0_npm= None)
GGL.fit()
np.linalg.norm(GGL.fit_[0]['beta'])

0
lr 0.5
lr 0.25
lr 0.125
lr 0.0625
lr 0.03125
1
lr 0.03125
2
lr 0.03125
converged 7.58


0.0

In [226]:
#repeat the covariate matrx m times

In [229]:
# r = 0 
# GGL = GradientGroupLasso(replicates[r].dg_M[:1], replicates[r].df_M[:1], np.asarray([.8655]), np.asarray([0.]), 500,.5, 1e-14, beta0_npm= None)

In [92]:
replicates[r].df_M[:1][:,:1,:3]

array([[[ 0.00494647,  0.01348774, -0.1578278 ]]])

In [95]:
replicates[r].df_M[:1][:,:1,:3] * replicates[r].dg_M[:1][:,:1][:,:,:1]

array([[[-0.00522825, -0.01425607,  0.16681853]]])

In [64]:
a = np.einsum('n d m, n d p -> p' ,replicates[r].df_M[:2][:,:1,:1] , replicates[r].dg_M[:2][:,:1][:,:,:1])

In [65]:
b= np.einsum('n d m, n d p -> p' ,replicates[r].df_M[1:2][:,:1,:1] , replicates[r].dg_M[1:2][:,:1][:,:,:1])

In [71]:
c= np.einsum('n d m, n d p -> p' ,replicates[r].df_M[0:1][:,:1,:1] , replicates[r].dg_M[0:1][:,:1][:,:,:1])

In [101]:
b #this is the 0:2 lambda !!!!!!!!!!!

array([-0.17835414])

In [102]:
np.einsum('n d m, n d p -> n p m ' ,replicates[r].df_M[:2][:,:1,:1] , replicates[r].dg_M[:2][:,:1][:,:,:1])

array([[[-0.00522825]],

       [[-0.17835414]]])

In [97]:
np.einsum('n d m, n d p -> n p m ' ,replicates[r].df_M[:2][:,:1,:3] , replicates[r].dg_M[:2][:,:1][:,:,:1])

array([[[-0.00522825, -0.01425607,  0.16681853]],

       [[-0.17835414, -0.05975958, -0.23167237]]])

In [123]:
np.linalg.norm([-0.00522825,-0.17835414])

0.17843075366427757

In [124]:
np.linalg.norm(np.einsum('n d m, n d p -> n p m ' ,replicates[r].df_M[:2][:,:1,:3] , replicates[r].dg_M[:2][:,:1][:,:,:1]))

0.34221701638579616

In [126]:
#max over n
r = 0 
GGL = GradientGroupLasso(replicates[r].dg_M[:2][:,:1][:,:,:1], replicates[r].df_M[:2][:,:1,:3], np.asarray([.3423]), np.asarray([0.]), 500,.5, 1e-14, beta0_npm= None)
GGL.fit()
np.linalg.norm(GGL.fit_[0]['beta'])

0
lr 0.5
lr 0.25
lr 0.125
1
lr 0.125
2
lr 0.125
3
lr 0.125
4
lr 0.125
5
lr 0.125
6
lr 0.125
7
lr 0.125
converged 0.3423


0.0

In [130]:
np.linalg.norm(np.einsum('n d m, n d p -> n p m ' ,replicates[r].df_M[:2][:,:1,:3] , replicates[r].dg_M[:2][:,:1][:,:,:1]))

0.34221701638579616

In [131]:
np.linalg.norm(np.einsum('n d m, n d p -> n p m ' ,replicates[r].df_M[:2][:,:1,:3] , replicates[r].dg_M[:2][:,:1][:,:,1:2]))

0.30578416710841466

In [141]:
np.linalg.norm(np.einsum('n d m, n d p -> n p m ' ,replicates[r].df_M[:2][:,:1,:3] , replicates[r].dg_M[:2][:,:1][:,:,:5]), axis = tuple([0,2]))

array([0.34221702, 0.30578417, 0.30809189, 0.30619914, 0.2774736 ])

In [133]:
#max over n
r = 0 
GGL = GradientGroupLasso(replicates[r].dg_M[:2][:,:1][:,:,:2], replicates[r].df_M[:2][:,:1,:3], np.asarray([.343]), np.asarray([0.]), 500,.5, 1e-14, beta0_npm= None)
GGL.fit()
np.linalg.norm(GGL.fit_[0]['beta'])

0
lr 0.5
lr 0.25
lr 0.125
lr 0.0625
1
lr 0.0625
2
lr 0.0625
3
lr 0.0625
4
lr 0.0625
5
lr 0.0625
6
lr 0.0625
7
lr 0.0625
8
lr 0.0625
9
lr 0.0625
converged 0.343


0.0

Our objective function is
\begin{eqnarray*}
\| y_{ik} - \sum_j \beta_{ijk} x_{ij} \|_2^2 = \lambda \sum_j  \| \beta_{ijk} \|_2^2 \\
\nabla_{\beta = 0} \| y_{ik} - \sum_j \beta_{ijk} x_{ij} \|_2^2 = y_{ik} x_{ij}\\
y_{ik} x_{ij} = \lambda
\end{eqnarray*}

The maximum lambda value is 
\begin{eqnarray*}
\lambda_{max} = \max_j \| y_{ik}^T x_{ij} \|_2
\end{eqnarray*}


In [55]:
-1.05696544 * 0.00494647 + 2.62957466*-0.06782623

-0.1835823835313286

In [41]:
replicates[r].dg_M[:2][:,:1][:,:,:1]

array([[[-1.05696544]],

       [[ 2.62957466]]])

In [40]:
replicates[r].df_M[:2][:,:1,:1]

array([[[ 0.00494647]],

       [[-0.06782623]]])

In [ ]:
#both are improved by moving in the same direction.... larger lambda

In [83]:
r = 0 
GGL = GradientGroupLasso(replicates[r].dg_M[:2][:,:1][:,:,:3], replicates[r].df_M[:2][:,:1,:1], np.asarray([.178]), np.asarray([0.]), 500,.5, 1e-14, beta0_npm= None)
GGL.fit()
np.linalg.norm(GGL.fit_[0]['beta'])

0
lr 0.5
lr 0.25
lr 0.125
lr 0.0625
lr 0.03125
1
lr 0.03125
2
lr 0.03125
3
lr 0.03125
4
lr 0.03125
5
lr 0.03125
6
lr 0.03125
7
lr 0.03125
8
lr 0.03125
9
lr 0.03125
10
lr 0.03125
11
lr 0.03125
12
lr 0.03125
13
lr 0.03125
14
lr 0.03125
15
lr 0.03125
16
lr 0.03125
17
lr 0.03125
18
lr 0.03125
19
lr 0.03125
converged 0.178


1.3461122002021633e-05

In [52]:
r = 0 
GGL = GradientGroupLasso(replicates[r].dg_M[:2][:,:1][:,:,:1], replicates[r].df_M[:2][:,:1,:1], np.asarray([.1785]), np.asarray([0.]), 2000,.5, 1e-14, beta0_npm= None)
GGL.fit()
np.linalg.norm(GGL.fit_[0]['beta'])

0
lr 0.5
lr 0.25
lr 0.125
1
lr 0.125
2
lr 0.125
3
lr 0.125
4
lr 0.125
5
lr 0.125
6
lr 0.125
7
lr 0.125
8
lr 0.125
9
lr 0.125
10
lr 0.125
11
lr 0.125
converged 0.1785


0.0

In [286]:
r = 0 
GGL = GradientGroupLasso(replicates[r].dg_M[:1][:,:1][:,:,:3], replicates[r].df_M[:1][:,:1,:1], np.asarray([.00522]), np.asarray([0.]), 500,.5, 1e-14, beta0_npm= None)
GGL.fit()
np.linalg.norm(GGL.fit_[0]['beta'])

0
lr 0.5
lr 0.25
1
lr 0.25
2
lr 0.25
3
lr 0.25
4
lr 0.25
5
lr 0.25
6
lr 0.25
7
lr 0.25
8
lr 0.25
9
lr 0.25
10
lr 0.25
11
lr 0.25
12
lr 0.25
13
lr 0.25
14
lr 0.25
15
lr 0.25
16
lr 0.25
17
lr 0.25
18
lr 0.25
19
lr 0.25
20
lr 0.25
21
lr 0.25
22
lr 0.25
23
lr 0.25
converged 0.00522


2.0615798366366376e-06

In [290]:
r = 0 
GGL = GradientGroupLasso(replicates[r].dg_M[:1][:,:1][:,:,:3], replicates[r].df_M[:1][:,:1,:1], np.asarray([.00523]), np.asarray([0.]), 500,.5, 1e-14, beta0_npm= None)
GGL.fit()
np.linalg.norm(GGL.fit_[0]['beta'])

0
lr 0.5
1
lr 0.5
2
lr 0.5
3
lr 0.5
4
lr 0.5
5
lr 0.5
lr 0.25
6
lr 0.25
7
lr 0.25
8
lr 0.25
9
lr 0.25
10
lr 0.25
11
lr 0.25
12
lr 0.25
13
lr 0.25
14
lr 0.25
15
lr 0.25
16
lr 0.25
17
lr 0.25
18
lr 0.25
19
lr 0.25
20
lr 0.25
21
lr 0.25
22
lr 0.25
23
lr 0.25
24
lr 0.25
25
lr 0.25
26
lr 0.25
27
lr 0.25
28
lr 0.25
29
lr 0.25
30
lr 0.25
31
lr 0.25
32
lr 0.25
33
lr 0.25
34
lr 0.25
35
lr 0.25
36
lr 0.25
37
lr 0.25
38
lr 0.25
39
lr 0.25
40
lr 0.25
41
lr 0.25
42
lr 0.25
43
lr 0.25
44
lr 0.25
45
lr 0.25
46
lr 0.25
47
lr 0.25
48
lr 0.25
49
lr 0.25
50
lr 0.25
51
lr 0.25
52
lr 0.25
53
lr 0.25
54
lr 0.25
55
lr 0.25
56
lr 0.25
57
lr 0.25
58
lr 0.25
59
lr 0.25
60
lr 0.25
61
lr 0.25
62
lr 0.25
63
lr 0.25
64
lr 0.25
65
lr 0.25
66
lr 0.25
67
lr 0.25
68
lr 0.25
69
lr 0.25
70
lr 0.25
71
lr 0.25
72
lr 0.25
73
lr 0.25
74
lr 0.25
75
lr 0.25
76
lr 0.25
77
lr 0.25
78
lr 0.25
79
lr 0.25
80
lr 0.25
81
lr 0.25
82
lr 0.25
83
lr 0.25
84
lr 0.25
85
lr 0.25
86
lr 0.25
87
lr 0.25
88
lr 0.25
89
lr 0.25
90
lr 0.25
91
lr 0

0.0

In [237]:
#m makes a difference... p may not make a difference, depends on max

In [267]:
r = 0 
GGL = GradientGroupLasso(replicates[r].dg_M[:2][:,:1][:,:,:3], replicates[r].df_M[:2][:,:1,:1], np.asarray([.179]), np.asarray([0.]), 500,.5, 1e-14, beta0_npm= None)
GGL.fit()
np.linalg.norm(GGL.fit_[0]['beta'])

0
lr 0.5
lr 0.25
lr 0.125
lr 0.0625
lr 0.03125
1
lr 0.03125
2
lr 0.03125
3
lr 0.03125
4
lr 0.03125
5
lr 0.03125
6
lr 0.03125
7
lr 0.03125
8
lr 0.03125
9
lr 0.03125
10
lr 0.03125
11
lr 0.03125
12
lr 0.03125
13
lr 0.03125
14
lr 0.03125
15
lr 0.03125
16
lr 0.03125
17
lr 0.03125
18
lr 0.03125
19
lr 0.03125
20
lr 0.03125
21
lr 0.03125
22
lr 0.03125
23
lr 0.03125
24
lr 0.03125
25
lr 0.03125
26
lr 0.03125
27
lr 0.03125
28
lr 0.03125
converged 0.179


0.0

In [268]:
np.linalg.norm(GGL.fit_[0]['beta'])

0.0

In [181]:
replicates[r].df_M[:1]

array([[[ 0.00494647,  0.01348774, -0.1578278 ],
        [-0.11964988,  0.1085518 , -0.06214833]]])

In [117]:
ys = experiment.construct_Y_js(replicates[r].df_M[:1], 2)
xs,gs = experiment.construct_X_js(np.swapaxes(replicates[r].dg_M[:1],1,2))
sums = np.zeros(12)
for i in range(36):
    sums[i%12] += (xs.transpose() @ ys)[i]
    
np.abs(sums).max() / np.sqrt(1*3)

0.3173332412132229

In [90]:
import spams

In [ ]:
p = len(np.unique(groups))
lambdas = np.asarray(lambdas, dtype=np.float64)
yadd = np.expand_dims(ys, 1)
groups = np.asarray(groups, dtype=np.int32) + 1
W0 = np.zeros((xs.shape[1], yadd.shape[1]), dtype=np.float32)
Xsam = np.asfortranarray(xs, dtype=np.float32)
Ysam = np.asfortranarray(yadd, dtype=np.float32)
coeffs = np.zeros((len(lambdas), q, n, p))

# alpha = spams.fistaFlat(Xsam,Dsam2,alpha0sam,ind_groupsam,lambda1 = lambdas[i],mode = mode,itermax = itermax,tol = tol,numThreads = numThreads, regul = "group-lasso-l2")
# spams.fistaFlat(Y,X,W0,TRUE,numThreads = 1,verbose = TRUE,lambda1 = 0.05, it0 = 10, max_it = 200,L0 = 0.1, tol = 1e-3, intercept = FALSE,pos = FALSE,compute_gram = TRUE, loss = 'square',regul = 'l1')
#output = spams.fistaFlat(Ysam, ssp.csc_matrix(Xsam), W0, True, groups=groups, numThreads=-1, verbose=True,
output = spams.fistaFlat(Ysam, Xsam, W0, True, groups=groups, numThreads=-1, verbose=True,
                         lambda1=lambdas[i], it0=100, max_it=itermax, L0=0.5, tol=tol, intercept=False,
                         pos=False, compute_gram=True, loss='square', regul='group-lasso-l2', ista=False,
                         subgrad=False, a=.1, b=1000)#b = 1000, a =.1
coeffs[i, :, :, :] = np.reshape(output[0], (q, n, p))
